# Flights update

## 1&nbsp;Flights function definition

In [ ]:
def flight_new_input(input):

    # 01: Import libraries
    import os
    import gc
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup
    from datetime import datetime, timedelta
    from sqlalchemy import create_engine, text

    # 02: Transform possible inputs into list of cities
    if (type(input) == int) | (type(input) == float):
        city_list = list()
    elif type(input) == str:
        city_list = [input]
    elif type(input) == pd.core.series.Series:
        city_list = list(input)
    elif type(input) == pd.core.frame.DataFrame:
        city_list = list(input['city'])
    elif type(input) == list:
        city_list = input.copy()
    elif type(input) == set:
        city_list = list(input)
    elif type(input) == dict:
        if 'city' in input.keys():
            if type(input['city']) == list:
                city_list = input['city']
            elif type(input['city']) == str:
                city_list = [input['city']]
        else:
            city_list = list(input.keys())

    # 03: Database reading
    schema = 'gans'
    host = os.getenv('Cloud_MySQL_HOST')
    user = 'root'
    password = os.getenv('Cloud_MySQL_API_KEY')
    port = 3306

    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    engine = create_engine(connection_string)

    city_sql = pd.read_sql(text('SELECT * FROM city'), con=engine.connect())
    city_info_static_sql = pd.read_sql(text('SELECT * FROM city_info_static'), con=engine.connect())
    city_info_static_sql.drop('city_info_static_id', axis=1, inplace=True)
    weather_sql_df = pd.read_sql_query(text('SELECT * FROM weather'), con=engine.connect())

    city_list_default = list(city_sql['city_name'])

    # 04: Check if the cities from input are in the database
    if city_list == []:# not in locals():
        city_list = city_list_default.copy()

    city_included = [city for city in city_list if city in city_sql['city_name'].values]
    city_not_included = [city for city in city_list if city not in city_sql['city_name'].values]
    city_excluded = []

    # 05: Create new data frames for new cities
    city_info_static_new_df = pd.DataFrame(columns=['city_id', 'country', 'latitude', 'longitude', 'latitude_num', 'longitude_num', 'website'])
    city_info_static_new_df['latitude_num'] = pd.to_numeric(city_info_static_new_df['latitude_num'], errors='coerce')
    city_info_static_new_df['longitude_num'] = pd.to_numeric(city_info_static_new_df['longitude_num'], errors='coerce')
    city_info_time_new_df = pd.DataFrame(columns=['city_id', 'area_city_km2', 'population', 'timestamp'])
    city_info_time_new_df['area_city_km2'] = pd.to_numeric(city_info_time_new_df['area_city_km2'], errors='coerce')
    city_info_time_new_df['population'] = pd.to_numeric(city_info_time_new_df['population'], errors='coerce')
    city_info_time_new_df['timestamp'] = pd.to_datetime(city_info_time_new_df['timestamp'], errors='coerce')

    # 06 function: Grabbing data from Wikipedia for cities not in database
    cities = []
    for city in range(len(city_not_included)):
        url = 'https://en.wikipedia.org/wiki/' + city_not_included[city]
        response = requests.get(url)
        infobox = BeautifulSoup(response.content, 'html.parser').find(class_='infobox ib-settlement vcard')
        
        try:
            latitude_element = infobox.find(class_='latitude')
            longitude_element = infobox.find(class_='longitude')

            cities.append(city_not_included[city])
            city_info_static_new_df.loc[city, 'country'] = infobox.find(class_='infobox-data').get_text()
            city_info_static_new_df.loc[city, 'latitude'] = infobox.find(class_='latitude').get_text()
            city_info_static_new_df.loc[city, 'longitude'] = infobox.find(class_='longitude').get_text()
            city_info_static_new_df.loc[city, 'latitude_num'] = pd.to_numeric(infobox.select('span.geo')[0].get_text().split(";")[0])
            city_info_static_new_df.loc[city, 'longitude_num'] = pd.to_numeric(infobox.select('span.geo')[0].get_text().split(";")[1])
            city_info_static_new_df.loc[city, 'website'] = infobox.find(string="Website").find_next("td").get_text()
            city_info_static_new_df.reset_index(drop=True, inplace=True)
            city_info_time_new_df.loc[city, 'area_city_km2'] = pd.to_numeric(infobox.find(string='Area').find_next(class_='infobox-data').get_text().split('km2')[0][:-1].replace(',', ''))
            city_info_time_new_df.loc[city, 'population'] = pd.to_numeric(infobox.find(string='Population').find_next(class_='infobox-data').get_text().replace(',', ''))
            city_info_time_new_df.loc[city, 'timestamp'] = pd.to_datetime(datetime.now()).strftime('%Y-%m-%d %H:%M:%S')
            city_info_time_new_df.reset_index(drop=True, inplace=True)
        except AttributeError:
            continue

    # 07: Filling in new cities into SQL, grabbing their new IDs and filling in the info-dataframes with the IDs and info
    for city in range(len(cities)):
        if cities[city] not in city_sql['city_name'].values:
            df = pd.DataFrame([cities[city]], columns=['city_name'])
            df.to_sql('city',
                if_exists='append',
                con=connection_string,
                index=False)
            id = pd.read_sql(text(f'SELECT city_id FROM city WHERE city_name = "{cities[city]}"'), con=engine.connect())['city_id'][0]
            city_info_static_new_df.loc[city, 'city_id'] = id
            city_info_time_new_df.loc[city, 'city_id'] = id

            dft = city_info_time_new_df.loc[[city]]
            dft.to_sql('city_info_time',
                if_exists='append',
                con=connection_string,
                index=False)
            
            dfs = city_info_static_new_df.loc[[city]]
            dfs.to_sql('city_info_static',
                if_exists='append',
                con=connection_string,
                index=False)

    # 08: Get airports for cities from API
    airport_sql_df = pd.read_sql(text('SELECT * FROM airport'), con=engine.connect())
    city_sql_df = pd.read_sql(text('SELECT * FROM city'), con=engine.connect())
    city_info_static_sql_df = pd.read_sql(text('SELECT * FROM city_info_static'), con=engine.connect())

    selected_city = city_sql_df.loc[city_sql_df['city_name'].isin(city_list), 'city_id']
    geo_selected_city = city_info_static_sql_df.loc[city_info_static_sql_df['city_id'].isin(selected_city), ['city_id', 'latitude_num', 'longitude_num']].reset_index(drop=True)#[['city_id', 'latitude_num', 'longitude_num']]

    airport_new_df = pd.DataFrame(columns=['city_id', 'icao', 'iata', 'airport_name', 'airport_name_short'])

    for index, value in enumerate(geo_selected_city.reset_index(drop=True)['city_id']):
        url = f'https://aerodatabox.p.rapidapi.com/airports/search/location/'
        querystring = {
            'lat':f'{geo_selected_city.loc[index, '''latitude_num''']}'
            ,'lon':f'{geo_selected_city.loc[index, '''longitude_num''']}'
            ,'radiusKm':'50'
            ,'limit':'10','withFlightInfoOnly':'true'
        }
        headers = {
            "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com",
            "X-RapidAPI-Key": os.getenv('Rapid_API_KEY')
        }
        response = requests.request("GET", url, headers=headers, params=querystring)
        if response.json()['count'] != 0:
            response_normalized = pd.json_normalize(response.json()['items'])[['icao', 'iata', 'name', 'shortName']]
            response_normalized.rename(columns={'name': 'airport_name', 'shortName': 'airport_name_short'}, inplace=True)
            response_normalized['city_id'] = value
            airport_new_df = pd.concat([airport_new_df, response_normalized])
    airport_new_df = airport_new_df.loc[~airport_new_df['iata'].isin(airport_sql_df['iata'])].reset_index(drop=True)#[['city_id', 'icao', 'iata', 'airport_name', 'airport_name_short']]

    # 09 Filling in new airports into SQL database
    for airport in range(len(airport_new_df)):
        dfa = airport_new_df.loc[[airport]]
        dfa.to_sql('airport',
            if_exists='append',
            con=connection_string,
            index=False)
    
    # 10 Get flights from API
    airport_sql_df = pd.read_sql(text('SELECT * FROM airport'), con=engine.connect())
    city_sql_df = pd.read_sql(text('SELECT * FROM city'), con=engine.connect())
    flight_sql_df = pd.read_sql(text('SELECT * FROM flight'), con=engine.connect())

    selected_city = city_sql_df.loc[city_sql_df['city_name'].isin(city_list), 'city_id']
    selected_city_airport = airport_sql_df.loc[airport_sql_df['airport_id'].isin(selected_city), ['airport_id', 'iata']].reset_index(drop=True)#.drop(['index'], axis=1)

    from datetime import datetime
    flight_api_df = pd.DataFrame(columns=flight_sql_df.columns.drop('flight_id'))
    flight_api_df['arrival_scheduledTime_local'] = pd.to_datetime(flight_api_df['arrival_scheduledTime_local'], errors='coerce')
    flight_api_df['arrival_revisedTime_local'] = pd.to_datetime(flight_api_df['arrival_revisedTime_local'], errors='coerce')
    tomorrow = datetime.now() + timedelta(days=1)
    day_after_tomorrow = datetime.now() + timedelta(days=2)

    for index, row in selected_city_airport.iterrows():
        url = f'https://aerodatabox.p.rapidapi.com/flights/airports/iata/{row['iata']}/{tomorrow.strftime('%Y-%m-%dT00:00')}/{tomorrow.strftime('%Y-%m-%dT12:00')}'
        querystring = {"withLeg":"true","direction":"Both","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"true"}
        headers = {'x-rapidapi-key': os.getenv('Rapid_API_KEY'), 'x-rapidapi-host': 'aerodatabox.p.rapidapi.com'}
        response = requests.request("GET", url, headers=headers, params=querystring)
        if response.status_code == 200:
            response_normalized = pd.json_normalize(response.json()['arrivals'])
            response_normalized = response_normalized.drop_duplicates(subset=['number', 'arrival.scheduledTime.local'])
            if 'arrival.revisedTime.local' not in response_normalized.columns:
                response_normalized['arrival.revisedTime.local'] = response_normalized['arrival.scheduledTime.local']
            response_normalized['airline.iata'] = response_normalized.get('airline.iata', None)
            response_normalized['arrival.revisedTime.local'] = response_normalized['arrival.revisedTime.local'].fillna(response_normalized['arrival.scheduledTime.local'])
            response_normalized['arrival.scheduledTime.local'] = pd.to_datetime(response_normalized['arrival.scheduledTime.local'].str[0:16])
            response_normalized['arrival.revisedTime.local'] = pd.to_datetime(response_normalized['arrival.revisedTime.local'].str[0:16])
            response_normalized['airport_id'] = row[0]
            response_normalized['bound'] = 'arrival'
            response_normalized['flight_key'] = response_normalized['number'] + '|' + response_normalized['arrival.scheduledTime.local'].dt.strftime('%Y-%m-%d %H:%M')
            response_normalized.rename(columns={
                'departure.airport.icao': 'departure_airport_icao'
                , 'departure.airport.iata': 'departure_airport_iata'
                , 'departure.airport.name': 'departure_airport_name'
                , 'arrival.scheduledTime.local': 'arrival_scheduledTime_local'
                , 'arrival.revisedTime.local': 'arrival_revisedTime_local'
                , 'aircraft.model': 'aircraft_model'
                , 'airline.name': 'airline_name'
                , 'airline.icao': 'airline_icao'
                , 'airline.iata': 'airline_iata'
            }, inplace=True)
            response_normalized = response_normalized[flight_api_df.columns]
            flight_api_df = pd.concat([flight_api_df, response_normalized])

    for index, row in selected_city_airport.iterrows():
        url = f'https://aerodatabox.p.rapidapi.com/flights/airports/iata/{row['iata']}/{tomorrow.strftime('%Y-%m-%dT12:00')}/{day_after_tomorrow.strftime('%Y-%m-%dT00:00')}'
        querystring = {"withLeg":"true","direction":"Both","withCancelled":"true","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"true"}
        headers = {'x-rapidapi-key': os.getenv('Rapid_API_KEY'), 'x-rapidapi-host': 'aerodatabox.p.rapidapi.com'}
        response = requests.request("GET", url, headers=headers, params=querystring)
        if response.status_code == 200:
            response_normalized = pd.json_normalize(response.json()['arrivals'])
            response_normalized = response_normalized.drop_duplicates(subset=['number', 'arrival.scheduledTime.local'])
            if 'arrival.revisedTime.local' not in response_normalized.columns:
                response_normalized['arrival.revisedTime.local'] = response_normalized['arrival.scheduledTime.local']
            response_normalized['airline.iata'] = response_normalized.get('airline.iata', None)
            response_normalized['arrival.revisedTime.local'] = response_normalized['arrival.revisedTime.local'].fillna(response_normalized['arrival.scheduledTime.local'])
            response_normalized['arrival.scheduledTime.local'] = pd.to_datetime(response_normalized['arrival.scheduledTime.local'].str[0:16])
            response_normalized['arrival.revisedTime.local'] = pd.to_datetime(response_normalized['arrival.revisedTime.local'].str[0:16])
            response_normalized['airport_id'] = row[0]
            response_normalized['bound'] = 'arrival'
            response_normalized['flight_key'] = response_normalized['number'] + '|' + response_normalized['arrival.scheduledTime.local'].dt.strftime('%Y-%m-%d %H:%M')
            response_normalized.rename(columns={
                'departure.airport.icao': 'departure_airport_icao'
                , 'departure.airport.iata': 'departure_airport_iata'
                , 'departure.airport.name': 'departure_airport_name'
                , 'arrival.scheduledTime.local': 'arrival_scheduledTime_local'
                , 'arrival.revisedTime.local': 'arrival_revisedTime_local'
                , 'aircraft.model': 'aircraft_model'
                , 'airline.name': 'airline_name'
                , 'airline.icao': 'airline_icao'
                , 'airline.iata': 'airline_iata'
            }, inplace=True)
            response_normalized = response_normalized[flight_api_df.columns]
            flight_api_df = pd.concat([flight_api_df, response_normalized])

    flight_existing_df = flight_api_df.loc[flight_api_df['flight_key'].isin(flight_sql_df['flight_key'])].reset_index(drop=True)
    flight_new_df = flight_api_df.loc[~flight_api_df['flight_key'].isin(flight_sql_df['flight_key'])].reset_index(drop=True)

    # 12 Filling in the flights into SQL
    for flight in range(len(flight_new_df)):
        dff = flight_new_df.loc[[flight]]
        dff.to_sql('flight',
            if_exists='append',
            con=connection_string,
            index=False)
    
    # 13 Updating existing values (not checking if anything changed)
    for _, row in flight_existing_df.iterrows():
        with engine.connect() as connection:
            sql = text(
                '''
                UPDATE
                    flight
                SET
                    airport_id = :airport_id
                    , bound = :bound
                    , status = :status
                    , departure_airport_icao = :departure_airport_icao
                    , departure_airport_iata = :departure_airport_iata
                    , departure_airport_name = :departure_airport_name
                    , arrival_revisedTime_local = :arrival_revisedTime_local
                    , aircraft_model = :aircraft_model
                    , airline_name = :airline_name
                    , airline_icao = :airline_icao
                    , airline_iata = :airline_iata
                WHERE flight_key = :flight_key
                ''')
            connection.execute(sql, {
                'airport_id': row['airport_id']
                , 'bound': row['bound']
                , 'status': row['status']
                , 'departure_airport_icao': row['departure_airport_icao'] if not pd.isna(row['departure_airport_icao']) else None
                , 'departure_airport_iata': row['departure_airport_iata'] if not pd.isna(row['departure_airport_iata']) else None
                , 'departure_airport_name': row['departure_airport_name'] if not pd.isna(row['departure_airport_name']) else None
                , 'arrival_revisedTime_local': row['arrival_revisedTime_local']
                , 'aircraft_model': row['aircraft_model'] if not pd.isna(row['aircraft_model']) else None
                , 'airline_name': row['airline_name'] if not pd.isna(row['airline_name']) else None
                , 'airline_icao': row['airline_icao'] if not pd.isna(row['airline_icao']) else None
                , 'airline_iata': row['airline_iata'] if not pd.isna(row['airline_iata']) else None
                , 'flight_key': row['flight_key']
            })
            connection.commit()
    
    # 14: Cleaning variables once they're no longer needed
    del city_sql, city_info_static_sql, weather_sql_df, city_info_static_new_df, city_info_time_new_df
    del airport_sql_df, airport_new_df
    del flight_api_df, flight_new_df, flight_existing_df
    del selected_city, selected_city_airport, geo_selected_city
    del cities, city_list, city_list_default, city_included, city_not_included, city_excluded
    del response, response_normalized, querystring, url
    del host, schema, user, password, port, connection_string
    engine.dispose()
    gc.collect()

## 2&nbsp;Input

In [3]:
# List the cities for which you need to collect flights.
city_list = ['Berlin', 'Aachen', 'Hamburg', 'Hannover', 'Bangkok']

## 3&nbsp;Run the function

In [4]:
flight_new_input(city_list)

## 4&nbsp;Results

In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine, text

schema = 'gans'
host = os.getenv('Cloud_MySQL_HOST')
user = 'root'
password = os.getenv('Cloud_MySQL_API_KEY')
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
engine = create_engine(connection_string)

flight_sql_df = pd.read_sql(text('SELECT * FROM flight'), con=engine.connect())

engine.dispose()
del schema, host, user, password, port, connection_string

flight_sql_df